# Experiments:
The experiments are done on Traffic Signs classification case study. Signs classification is a component of autonomous
vehicles and therefore its resistance to adversarial attacks is highly crucial. We talk about that much more in the
project report.

The dataset is GTSRB (German Traffic Sign Recognition).
I downloaded the dataset from: https://github.com/tomlawrenceuk/GTSRB-Dataloader

In short the experiments are:
1) Apply PGD and FGSM attacks on a spatial convolutional neural network.
2) Comparing robustness of adversarial training using FGSM versus using PGD.
3) Universality of PGD attack - we show that using PGD adversarial training we cover any first order attack (we show
   resistance to some known adversarial attacks - not a theoretical proof)
4) Capacity and Adversarial Robustness (see more details in Experiment 4)


In [1]:
# import libraries:

import torch
import attacks
import configs
import datasets
import dls
import helper
import models
import trainer
import os
import shutil


Initialization:
    - export configs
    - validate paths
    - set random seed
    - configure device (GPU / CPU)
    - load datasets
    - create hyperparameters generators

In [2]:
# configs
experiment_configs = configs.TrafficSigns_experiments_configs
experiment_hps_sets = configs.TrafficSigns_experiments_hps
show_test_successful_attacks_plots = configs.show_test_successful_attacks_plots
save_test_successful_attacks_plots = configs.show_test_successful_attacks_plots

# paths existence validation and initialization
assert os.path.exists(configs.data_root_dir), "The dataset should be in ./data/GTSRB"
assert os.path.exists(os.path.join(configs.data_root_dir, "GTSRB")), "The dataset should be in ./data/GTSRB"
if not os.path.exists(configs.results_folder):
    os.mkdir(configs.results_folder)
if os.path.exists(configs.plots_folder):
    shutil.rmtree(configs.plots_folder)
    os.mkdir(configs.plots_folder)
if os.path.exists(configs.logger_path):
    os.remove(configs.logger_path)

# seed
if configs.seed is not None:
    # np.random.seed(configs.seed)
    torch.manual_seed(configs.seed)

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# get datasets
transform = experiment_configs["data_transform"]
_training_dataset = datasets.GTSRB(root_dir=configs.data_root_dir, train=True, transform=transform)
_testing_dataset = datasets.GTSRB(root_dir=configs.data_root_dir, train=False, transform=transform)

# create hyperparameters generators
net_training_hps_gen = helper.GridSearch(experiment_hps_sets["nets_training"])
fgsm_attack_hps_gen = helper.GridSearch(experiment_hps_sets["FGSM"])
pgd_attack_hps_gen = helper.GridSearch(experiment_hps_sets["PGD"])


## Experiment 1: PGD and FGSM attacks in practice.

In this experiment we will attack a network using PGD and FGSM attacks.
The experiment illustrates that PGD and FGSM attacks actually works also on GTSRB dataset.
We train and attack a Spatial Transformer Network which is invariant to geometrical
transformations (rotations and scaling).


TypeError: 'builtin_function_or_method' object is not iterable

## Experiment 2: Comparing defensing using FGSM versus using PGD
In this experiment we will build 2 robust networks to the same task as the paper suggest but one will be resistant to FGSM attack
and the second to PGD attack.

Then we will examine the following results of the paper:

1. We show that network 1 (i.e. trained to be resistant to FGSM attack) is resistant to FGSM but not to PGD. Therefore we get that:
 - Resistancy FGSM attack does not implies PGD resistency.
 - FGSM attack is not universal.
2. We show that network 2 (i.e. trained to be resistant to PGD attack) is resistant to both PGD and FGSM.

   In the next experiment we show that network 2 is resistant to some other attacks and this examine the universality of PGD (i.e. that resistancy to
   PGD implies resistancy to any other first order attack).

## Experiment 3: Universality of PGD attack
In this experiment we will attack the network that is trained to be resistant to PGD attack
with other attacks like: ...........


Note that we already saw in experiment 2 that this network is resistant to FGSM attack.

In [ ]:
attacks_lst = [attacks.FGSM, attacks.PGD] # TODO: add more attacks
pgd_resistance_results = helper.measure_resistance_on_test(pgd_resistant_mnist_net, _loss_fn, _test_dataset, attacks_lst plots_title="robust net built using PGD")


## Experiment 4: Capacity and Adversarial Robustness
In this experiment we will examine the following statements:

1. Capacity alone helps: High capacity models are more robust to adversarial attacks then low capacity models.
2. Weak models may fail to learn non-trivial classifiers: We show that we can't build (using the paper method) a robust model
   when the model is with low capacity. Specifically we will show that after training with the paper method to build a robust
   model we get an extremely underfitted model.
3. The value of the saddle point problem decreases as we increase the capacity
4. More capacity and stronger adversaries decrease transferability: we take transferred adversarial inputs and show that their
   gradients correlation with the source becomes less significant. More details on that experiment can be found in the project file.

In [ ]:
# visualize and stufff........